In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from scipy.stats import linregress
import scipy.stats as st
from pprint import pprint

# Import API key
from config import weather_api_key

In [ ]:
# "api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}"
# Lists to call
resort_name = # add resort list here
date = [] # Make this date updated
lat = []
long = []
temp_main = [] # Use this to get current temperature
feels_like = []
temp_min = []
temp_max = []
temp_description = []
hum = []
wind_speed = []
cloudiness = []

x = 0 

print('Beginning Data Retrieval')
print("-"*64)  
for resort in resorts:
    url = "http://api.openweathermap.org/data/2.5/weather?"
    units = 'metric'
    query_url = f'{url}q={resort}&appid={weather_api_key}'
    
    
    try:
        # Get metrics from api doc/response
        response = requests.get(query_url + city).json()
        x = x + 1
        print(f'Processing Record {x} | {city}')
        resort_name.append(response['name'])
        date.append((datetime.fromtimestamp(response['dt'])).strftime('%m/%d/%y'))
        lat.append(response['coord']['lat'])
        long.append(response['coord']['lon'])
        temp_main.append(response['main']['temp'])
        feels_like.append(response['main']['feels_like'])
        temp_min.append(response['main']['temp_min'])
        temp_max.append(response['main']['temp_max'])
        temp_description.append(response['weather']['description'])
        hum.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        cloudiness.append(response['clouds']['all'])
    except KeyError:
        print("City not found")

In [ ]:
# json output

# {
#   "coord": {
#     "lon": -122.08,
#     "lat": 37.39
#   },
#   "weather": [
#     {
#       "id": 800,
#       "main": "Clear",
#       "description": "clear sky",
#       "icon": "01d"
#     }
#   ],
#   "base": "stations",
#   "main": {
#     "temp": 282.55,
#     "feels_like": 281.86,
#     "temp_min": 280.37,
#     "temp_max": 284.26,
#     "pressure": 1023,
#     "humidity": 100
#   },
#   "visibility": 16093,
#   "wind": {
#     "speed": 1.5,
#     "deg": 350
#   },
#   "clouds": {
#     "all": 1
#   },
#   "dt": 1560350645,
#   "sys": {
#     "type": 1,
#     "id": 5122,
#     "message": 0.0139,
#     "country": "US",
#     "sunrise": 1560343627,
#     "sunset": 1560396563
#   },
#   "timezone": -25200,
#   "id": 420006353,
#   "name": "Mountain View",
#   "cod": 200
#   }                        

In [ ]:
# SNOW API
# base_url = http://api.weatherapi.com/v1